In [1]:

!pip install flask
import json
from flask import Flask, jsonify, request
import pandas as pd

  Using cached MarkupSafe-2.1.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (25 kB)
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
stock = pd.read_csv('medication.csv')
stock_low_threshold = 10


app = Flask(__name__)

# whole stock
@app.route('/stock', methods=['GET'])
def get_stock():
    return jsonify(stock.to_json(orient="records"))
# infos for a single medication by name or id
@app.route('/stock/<name>', methods=['GET'])
def get_medication(name):
  return jsonify(stock.loc[stock['name'] == name].to_json(orient="records"))
@app.route('/stock/id<id>', methods=['GET'])
def get_medication_by_id(id):
  return jsonify(stock.loc[stock['id'] == int(id)].to_json(orient="records"))


### the following routes are used in camunda

# reorder things
@app.route('/stock/order', methods=['GET'])
def order():
    # return everything low on stock and not ordered yet
    return_df = stock.loc[(stock['stock'] < stock_low_threshold) & (~stock['ordered'])][['name','stock']]
    stock.loc[(stock['stock'] < stock_low_threshold) & (~stock['ordered']), 'ordered'] = True
    if len(return_df) == 0:
        return jsonify({"reorder_started": False})
    
    return jsonify({"reorder_started": True})


# called when something is sold
# in theory it would make sense to start the reorderin in here, when things get low
@app.route('/stock/<name>/deduct', methods=['POST'])
def deduct_medication_amount(name):
    amount = json.loads(request.data)['amount']

    

    if name in stock['name'].values.tolist():
    
        amount_in_stock = int(stock.loc[stock['name']==name]['stock'].item())
        if amount_in_stock - amount >= 0:
            stock.loc[stock['name'] == name, "stock"] -= amount
            return jsonify({"message": "Success"}), 200
        else:
            return jsonify({"message": "Not enough left"}), 200

    # doing it this way, because adding a lot of medication would be tedious and ugly in the camunda
    return return jsonify({"message": "Medication not tracked"}), 200  

    
# restocking gets called from storage room when restocking is done
# it ends with a check if everything is still stocked up
@app.route('/stock/restock', methods=['POST'])
def restock_medication_amount():
    # request.data
    # {"aspirin": 4, ...}
    
    meds = json.loads(request.data)
    
    for name, amount in meds.items():
        if amount > 0:
            stock.loc[stock['name'] == name, "stock"] += amount
            stock.loc[stock['name'] == name, "ordered"] = False

    
    return jsonify({"message": "Success"}), 200

@app.route('/debug', methods=['POST'])
def debug_method():
    print(request.data)

    return jsonify({"message": "Debug"}), 200


app.run(host='0.0.0.0', port=8080)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=46d040bc-851a-49ab-a8fb-23c965475dff' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>